In [ ]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(lubridate)
install.packages("zoo")
library(zoo)


In [ ]:
rawCH4 <- read.table("input_raw.txt", sep = ",", stringsAsFactors = F, header = T)  
colnames(rawCH4)
head(rawCH4)
dim(rawCH4)
summary(rawCH4)

In [ ]:
# Multiply to get ppm
rawCH4$ch4ppm <- rawCH4$ch4 * 10000
rawCH4$co2ppm <- rawCH4$co2 * 10000

# Parse datetime
rawCH4$datetime <- as.POSIXct(rawCH4$date, format = "%Y/%d/%m %H:%M:%S")
rawCH4$date <- as.Date.character(rawCH4$date, "%Y/%d/%m")
head(rawCH4)

In [ ]:
#create an event per entrance to the AMS per day per animal
event_ch4 <- rawCH4 %>%    
  select(ch4ppm, co2ppm, cow, date, datetime) %>%
  mutate(eventID = 1 + cumsum(cow != lag(cow, default = first(cow))))

hist(event_ch4$eventID)
head(event_ch4)

In [ ]:
num_cows <- event_ch4 %>%
  summarise(unique_cows = n_distinct(cow))
 print(num_cows)

cow_counts <- rawCH4 %>%
  group_by(cow) %>%
  summarise(count = n())
head(cow_counts)

In [ ]:
########## calculating duration of the visit ##############
event_duration <- event_ch4 %>%
  group_by(eventID) %>%
  summarize(
    start_time = min(datetime),
    end_time = max(datetime),
    lenght = as.numeric(difftime(max(datetime), min(datetime), units = "secs"))
  )
head(event_duration)
summary(event_duration)
hist(event_duration$lenght)

In [ ]:
#Calculating how many events (visit) per cow 

event_counts <- event_ch4 %>%
  group_by(cow) %>%
  summarise(unique_events = n_distinct(eventID))

# View the result
print(event_counts)


In [ ]:
############ calculating the average of the lowest 3 or 5 measurements per visit to use it as background ########
event_5low <- event_ch4 %>%
  group_by(eventID) %>%
  arrange(ch4ppm) %>%
  slice_head(n = 5) %>%
  summarise(avg_ch4 = mean(ch4ppm, na.rm = TRUE))
head(event_5low)
summary(event_5low)

In [ ]:
event_3low <- event_ch4 %>%
  group_by(eventID) %>%
  arrange(ch4ppm) %>%
  slice_head(n = 3) %>%
  summarise(avg_ch4 = mean(ch4ppm, na.rm = TRUE))
head(event_3low)
summary(event_3low)

In [ ]:
############ calculating the 0.001 quantile per visit to use it as background ############
event_quant <- event_ch4 %>%
  group_by(eventID) %>%
  summarise(ch4_quantile_0_001 = quantile(ch4ppm, probs = 0.001, na.rm = TRUE))
head(event_quant)
summary(event_quant)

In [ ]:
############### calculating the mean for ch4 and co2 per visit #################
event_means <- event_ch4 %>%
  group_by(eventID) %>%
  summarise(
    mean_ch4 = mean(ch4ppm, na.rm = TRUE),
    mean_co2 = mean(co2ppm, na.rm = TRUE)
  )
head(event_means)
summary(event_means)

In [ ]:
############# calculating the mean for ch4 only for 60-300 sec per visit ######## losing one event
event_cut <- event_ch4 %>%
  # Trim any spaces from the TimeStamp column
  mutate(datetime = trimws(datetime)) %>%
  # Convert TimeStamp to datetime object
  mutate(datetime = ymd_hms(datetime)) %>%
  group_by(eventID) %>%
  # Calculate the time difference in seconds from the first TimeStamp per eventID
  mutate(time_diff_sec = as.numeric(difftime(datetime, min(datetime), units = "secs"))) %>%
  # Filter to keep only rows where the time difference is between 60 and 300 seconds
  filter(time_diff_sec >= 60 & time_diff_sec <= 300) %>%
  # Optionally, summarize the data (for example, calculating means of ch4 and co2)
  summarise(
    mean_ch4 = mean(ch4ppm, na.rm = TRUE),
    mean_co2 = mean(co2ppm, na.rm = TRUE)
  )
head(event_cut)
summary(event_cut)
dim(event_cut)

In [ ]:
#Printing cows numbers
cow_ids<- unique(event_ch4$cow)
print(cow_ids)


In [ ]:
#################### function to read the events of n cow ##################
# Function to get all events of a specific cow
get_unique_cow_event_ids <- function(cow_number) {
  cow_events <- subset(event_ch4, cow == cow_number)
  unique_event_ids <- unique(cow_events$eventID)
  return(unique_event_ids)
}

cow_number <- 72
unique_event_ids <- get_unique_cow_event_ids(cow_number)
print(unique_event_ids)


In [ ]:
############# lets plot an event of a cow   ####### 

# Choose a specific cow and event for the plot
cow_id <- 72
event_id <- 138

plot_data <- subset(event_ch4, cow == cow_id & eventID == event_id)

# Create ggplot plot
p <- ggplot(plot_data, aes(x = datetime, y = ch4ppm)) +
  geom_line() +
  geom_point() +
  labs(title = paste("CH4 Time Series for Cow", cow_id, "in Event", event_id),
       x = "Datetime",
       y = "Ch4") +
  theme_minimal()

# Explicitly print and assign a name to the plot
print(p)


In [ ]:
# Function to detect peaks
detectPeaks <- function(ch4ppm, windowSize = 5, threshold = 0.0005) {
  peaksIni <- c()
  peaksFin <- c()
  stage <- FALSE
  peak_decreasing <- FALSE
  
  for (i in 1:(length(ch4ppm) - windowSize)) {
    tempW <- ch4ppm[i:(i + windowSize - 1)]
    tempX <- 1:windowSize
    
    mean_tempW <- mean(tempW, na.rm = TRUE)
    mean_tempX <- mean(tempX, na.rm = TRUE)
    
    cov_xy <- sum((tempW - mean_tempW) * (tempX - mean_tempX), na.rm = TRUE)
    var_x <- sum((tempX - mean_tempX) * (tempX - mean_tempX), na.rm = TRUE)
    
    pendiente <- cov_xy / var_x
    
    if (pendiente > threshold && !stage) {
      peaksIni <- c(peaksIni, i)
      stage <- TRUE
      peak_decreasing <- FALSE
    } else if (stage && pendiente < -threshold) {
      peak_decreasing <- TRUE
    } else if (stage && peak_decreasing && pendiente > -threshold / 2) {
      peaksFin <- c(peaksFin, i)
      stage <- FALSE
      peak_decreasing <- FALSE
    } else if (length(peaksIni) == 0 && pendiente > threshold / 2) {
      # Handle case where no peaks have been detected yet
    } else if (i == (length(ch4ppm) - windowSize) && peak_decreasing) {
      peaksFin <- c(peaksFin, i)
    }
  }
  
  list(peaksIni = peaksIni, peaksFin = peaksFin)
}

# Apply the peak detection to the entire database
peak_det <- event_ch4 %>%
  group_by(eventID) %>%
  summarise(num_peaks = length(detectPeaks(ch4ppm)$peaksIni))

# Print the results
head(peak_det)
summary(peak_det)


In [ ]:
######################## EXERCISE #######################
#1)Calculate the num of peaks per  minute
    #HINT: Use merge, and lenght of event (visit) 

In [ ]:
# Function to detect peaks and calculate sum2maxpeaks
detectPeaksAndSum <- function(ch4ppm, windowSize = 5, threshold = 0.0005) {
  peaksIni <- c()
  peaksFin <- c()
  stage <- FALSE
  peak_decreasing <- FALSE
  
  for (i in 1:(length(ch4ppm) - windowSize)) {
    tempW <- ch4ppm[i:(i + windowSize - 1)]
    tempX <- 1:windowSize
    
    mean_tempW <- mean(tempW, na.rm = TRUE)
    mean_tempX <- mean(tempX, na.rm = TRUE)
    
    cov_xy <- sum((tempW - mean_tempW) * (tempX - mean_tempX), na.rm = TRUE)
    var_x <- sum((tempX - mean_tempX) * (tempX - mean_tempX), na.rm = TRUE)
    
    pendiente <- cov_xy / var_x
    
    if (pendiente > threshold && !stage) {
      peaksIni <- c(peaksIni, i)
      stage <- TRUE
      peak_decreasing <- FALSE
    } else if (stage && pendiente < -threshold) {
      peak_decreasing <- TRUE
    } else if (stage && peak_decreasing && pendiente > -threshold / 2) {
      peaksFin <- c(peaksFin, i)
      stage <- FALSE
      peak_decreasing <- FALSE
    } else if (length(peaksIni) == 0 && pendiente > threshold / 2) {
      # Handle case where no peaks have been detected yet
    } else if (i == (length(ch4ppm) - windowSize) && peak_decreasing) {
      peaksFin <- c(peaksFin, i)
    }
  }
  
  sum2maxpeaks <- 0
  if (length(peaksIni) > 0 && length(peaksFin) > 0) {
    for (j in 1:length(peaksIni)) {
      start <- peaksIni[j]
      end <- ifelse(j <= length(peaksFin), peaksFin[j], NA)
      if (!is.na(end) && end > start) {
        peak_values <- ch4ppm[start:end]
        max_values <- sort(peak_values, decreasing = TRUE)[1:2]
        avg_max_values <- mean(max_values)
        sum2maxpeaks <- sum2maxpeaks + avg_max_values
      }
    }
  }
  
  list(peaksIni = peaksIni, peaksFin = peaksFin, sum2maxpeaks = sum2maxpeaks)
}

# Apply the peak detection to the entire database
peak_phen <- event_ch4 %>%
  group_by(eventID) %>%
  summarise(
    num_peaks = {
      result <- detectPeaksAndSum(ch4ppm)
      length(result$peaksIni)
    },
    sum2maxpeaks = {
      result <- detectPeaksAndSum(ch4ppm)
      result$sum2maxpeaks
    }
  )

# Print the results
head(peak_phen)

In [ ]:
# Function to detect peaks and calculate sum2maxpeaks and area under the curve
detectPeaksAndSum <- function(ch4ppm, windowSize = 5, threshold = 0.0005) {
  peaksIni <- c()
  peaksFin <- c()
  stage <- FALSE
  peak_decreasing <- FALSE
  
  for (i in 1:(length(ch4ppm) - windowSize)) {
    tempW <- ch4ppm[i:(i + windowSize - 1)]
    tempX <- 1:windowSize
    
    mean_tempW <- mean(tempW, na.rm = TRUE)
    mean_tempX <- mean(tempX, na.rm = TRUE)
    
    cov_xy <- sum((tempW - mean_tempW) * (tempX - mean_tempX), na.rm = TRUE)
    var_x <- sum((tempX - mean_tempX) * (tempX - mean_tempX), na.rm = TRUE)
    
    pendiente <- cov_xy / var_x
    
    if (pendiente > threshold && !stage) {
      peaksIni <- c(peaksIni, i)
      stage <- TRUE
      peak_decreasing <- FALSE
    } else if (stage && pendiente < -threshold) {
      peak_decreasing <- TRUE
    } else if (stage && peak_decreasing && pendiente > -threshold / 2) {
      peaksFin <- c(peaksFin, i)
      stage <- FALSE
      peak_decreasing <- FALSE
    } else if (length(peaksIni) == 0 && pendiente > threshold / 2) {
      # Handle case where no peaks have been detected yet
    } else if (i == (length(ch4ppm) - windowSize) && peak_decreasing) {
      peaksFin <- c(peaksFin, i)
    }
  }
  
  sum2maxpeaks <- 0
  area_under_curve <- 0
  if (length(peaksIni) > 0 && length(peaksFin) > 0) {
    for (j in 1:length(peaksIni)) {
      start <- peaksIni[j]
      end <- ifelse(j <= length(peaksFin), peaksFin[j], NA)
      if (!is.na(end) && end > start) {
        peak_values <- ch4ppm[start:end]
        max_values <- sort(peak_values, decreasing = TRUE)[1:2]
        avg_max_values <- mean(max_values)
        sum2maxpeaks <- sum2maxpeaks + avg_max_values
        
        # Calculate area under the curve above ground level (assumed to be the minimum value in ch4)
        ground_level <- min(ch4ppm)
        area_under_curve <- area_under_curve + sum(peak_values - ground_level)
      }
    }
  }
  
  list(peaksIni = peaksIni, peaksFin = peaksFin, sum2maxpeaks = sum2maxpeaks, area_under_curve = area_under_curve)
}

# Apply the peak detection to the entire database
AUC <- event_ch4 %>%
  group_by(eventID) %>%
  summarise(
    num_peaks = {
      result <- detectPeaksAndSum(ch4ppm)
      length(result$peaksIni)
    },
    sum2maxpeaks = {
      result <- detectPeaksAndSum(ch4ppm)
      result$sum2maxpeaks
    },
    area_under_curve = {
      result <- detectPeaksAndSum(ch4ppm)
      result$area_under_curve
    }
  )

# Print the results
head(AUC)

In [ ]:
############################### now for the 'cut' visit ###########################

# Calculate the time difference in seconds from the first TimeStamp per eventID
event_ch42<- event_ch4 %>%
  group_by(eventID) %>%
  mutate(time_diff_sec = as.numeric(difftime(datetime, min(datetime), units = "secs"))) %>%
  ungroup()

# Function to detect peaks and calculate sum2maxpeaks and area under the curve
detectPeaksAndSum <- function(ch4ppm, windowSize = 5, threshold = 0.0005) {
  peaksIni <- c()
  peaksFin <- c()
  stage <- FALSE
  peak_decreasing <- FALSE
  
  for (i in 1:(length(ch4ppm) - windowSize)) {
    tempW <- ch4ppm[i:(i + windowSize - 1)]
    tempX <- 1:windowSize
    
    mean_tempW <- mean(tempW, na.rm = TRUE)
    mean_tempX <- mean(tempX, na.rm = TRUE)
    
    cov_xy <- sum((tempW - mean_tempW) * (tempX - mean_tempX), na.rm = TRUE)
    var_x <- sum((tempX - mean_tempX) * (tempX - mean_tempX), na.rm = TRUE)
    
    pendiente <- cov_xy / var_x
    
    if (pendiente > threshold && !stage) {
      peaksIni <- c(peaksIni, i)
      stage <- TRUE
      peak_decreasing <- FALSE
    } else if (stage && pendiente < -threshold) {
      peak_decreasing <- TRUE
    } else if (stage && peak_decreasing && pendiente > -threshold / 2) {
      peaksFin <- c(peaksFin, i)
      stage <- FALSE
      peak_decreasing <- FALSE
    } else if (length(peaksIni) == 0 && pendiente > threshold / 2) {
      # Handle case where no peaks have been detected yet
    } else if (i == (length(ch4ppm) - windowSize) && peak_decreasing) {
      peaksFin <- c(peaksFin, i)
    }
  }
  
  sum2maxpeaks <- 0
  area_under_curve <- 0
  if (length(peaksIni) > 0 && length(peaksFin) > 0) {
    for (j in 1:length(peaksIni)) {
      start <- peaksIni[j]
      end <- ifelse(j <= length(peaksFin), peaksFin[j], NA)
      if (!is.na(end) && end > start) {
        peak_values <- ch4ppm[start:end]
        max_values <- sort(peak_values, decreasing = TRUE)[1:2]
        avg_max_values <- mean(max_values)
        sum2maxpeaks <- sum2maxpeaks + avg_max_values
        
        # Calculate area under the curve above ground level (assumed to be the minimum value in ch4)
        ground_level <- min(ch4ppm)
        area_under_curve <- area_under_curve + sum(peak_values - ground_level)
      }
    }
  }
  
  list(peaksIni = peaksIni, peaksFin = peaksFin, sum2maxpeaks = sum2maxpeaks, area_under_curve = area_under_curve)
}

# Apply the peak detection to the entire database, considering only data between 60 sec to 300 sec per eventID
peak_phen2 <- event_ch42 %>%
  filter(time_diff_sec >= 60 & time_diff_sec <= 300) %>%
  group_by(eventID) %>%
  summarise(
    num_peaks = {
      result <- detectPeaksAndSum(ch4ppm)
      length(result$peaksIni)
    },
    sum2maxpeaks = {
      result <- detectPeaksAndSum(ch4ppm)
      result$sum2maxpeaks
    },
    area_under_curve = {
      result <- detectPeaksAndSum(ch4ppm)
      result$area_under_curve
    }
  )

# Print the results
head(peak_phen2)
dim(peak_phen2)
summary(peak_phen2)

In [ ]:
#Adding moving average, play with the window sizes
detectPeaksAndSum <- function(ch4ppm, windowSize = 5, threshold = 0.0005) {
  peaksIni <- c()
  peaksFin <- c()
  stage <- FALSE
  peak_decreasing <- FALSE
  
  for (i in 1:(length(ch4ppm) - windowSize)) {
    tempW <- ch4ppm[i:(i + windowSize - 1)]
    tempX <- 1:windowSize
    
    mean_tempW <- mean(tempW, na.rm = TRUE)
    mean_tempX <- mean(tempX, na.rm = TRUE)
    
    cov_xy <- sum((tempW - mean_tempW) * (tempX - mean_tempX), na.rm = TRUE)
    var_x <- sum((tempX - mean_tempX) * (tempX - mean_tempX), na.rm = TRUE)
    
    pendiente <- cov_xy / var_x
    
    if (pendiente > threshold && !stage) {
      peaksIni <- c(peaksIni, i)
      stage <- TRUE
      peak_decreasing <- FALSE
    } else if (stage && pendiente < -threshold) {
      peak_decreasing <- TRUE
    } else if (stage && peak_decreasing && pendiente > -threshold / 2) {
      peaksFin <- c(peaksFin, i)
      stage <- FALSE
      peak_decreasing <- FALSE
    } else if (length(peaksIni) == 0 && pendiente > threshold / 2) {
      # Handle case where no peaks have been detected yet
    } else if (i == (length(ch4ppm) - windowSize) && peak_decreasing) {
      peaksFin <- c(peaksFin, i)
    }
  }
  
  sum2maxpeaks <- 0
  area_under_curve <- 0
  if (length(peaksIni) > 0 && length(peaksFin) > 0) {
    for (j in 1:length(peaksIni)) {
      start <- peaksIni[j]
      end <- ifelse(j <= length(peaksFin), peaksFin[j], NA)
      if (!is.na(end) && end > start) {
        peak_values <- ch4ppm[start:end]
        max_values <- sort(peak_values, decreasing = TRUE)[1:2]
        avg_max_values <- mean(max_values)
        sum2maxpeaks <- sum2maxpeaks + avg_max_values
        
        # Calculate area under the curve above ground level (assumed to be the minimum value in ch4)
        ground_level <- min(ch4ppm)
        area_under_curve <- area_under_curve + sum(peak_values - ground_level)
      }
    }
  }
  
  list(peaksIni = peaksIni, peaksFin = peaksFin, sum2maxpeaks = sum2maxpeaks, area_under_curve = area_under_curve)
}

# Function to calculate the average of the moving average of CH4 values
calculate_moving_avg <- function(ch4ppm, window_size) {
  moving_avg <- rollmean(ch4ppm, k = window_size, fill = NA)
  mean(moving_avg, na.rm = TRUE)
}

# Apply the peak detection to the entire database, considering only data between 60 sec to 300 sec per eventID
all_phen <- event_ch42 %>%
  filter(time_diff_sec >= 60 & time_diff_sec <= 300) %>%
  group_by(eventID) %>%
  summarise(
    num_peaks = {
      result <- detectPeaksAndSum(ch4ppm)
      length(result$peaksIni)
    },
    sum2maxpeaks = {
      result <- detectPeaksAndSum(ch4ppm)
      result$sum2maxpeaks
    },
    area_under_curve = {
      result <- detectPeaksAndSum(ch4ppm)
      result$area_under_curve
    },
    avg_moving_avg_ch4 = calculate_moving_avg(ch4ppm, window_size = 10) # Change window_size as needed
  )

# Print the results
head(all_phen)
dim(all_phen)
summary(all_phen)

In [ ]:
#calculate all the phenotypes with cut and not cut and calculate correlations among the phenotypes
#HINT1: use merge or join
merged<- merge(event_5low, event_quant, by="eventID")
head(merged)
merged2<- left_join(event_5low, event_3low, by="eventID")
head(merged2)
#HINT2: use cor and choose the method kendall, spearman or pearson
cor(event_5low$avg_ch4, event_3low$avg_ch4, method="kendall", use="complete.obs")
cor(event_means$mean_ch4, event_means$mean_co2, method="pearson", use="complete.obs")